In [ ]:
import torch
print(torch.__version__)  # Kiểm tra phiên bản PyTorch
print(torch.cuda.is_available())  # Kiểm tra xem PyTorch có nhận GPU không
print(torch.version.cuda)  # Kiểm tra phiên bản CUDA mà PyTorch đang sử dụng
print(torch.cuda.get_device_name(0))  # Kiểm tra tên GPU


2.5.1+cu121
True
12.1
NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
torch.manual_seed(42)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix




In [8]:
# Dataset path
dataset_path = './Multi Cancer/Cervical Cancer'
cervix_categories = ['cervix_dyk', 'cervix_koc', 'cervix_mep', 'cervix_pab', 'cervix_sfi']

In [9]:
# Data preparation
filepath = []
labels = []
for category in cervix_categories:
    category_path = os.path.join(dataset_path, category)
    for image_name in os.listdir(category_path):
        filepath.append(os.path.join(category_path, image_name))
        labels.append(category)

# Convert to DataFrame
df = pd.DataFrame({'filepath': filepath, 'labels': labels})

# Encode labels
label_to_idx = {label: idx for idx, label in enumerate(cervix_categories)}
df['label_idx'] = df['labels'].map(label_to_idx)


In [10]:
# Train-validation-test split
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label_idx'])
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label_idx'])

# Image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
# Custom Dataset class
class CervicalCancerDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['filepath']
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToPILImage()(image)
        label = self.dataframe.iloc[idx]['label_idx']
        if self.transform:
            image = self.transform(image)
        return image, label

In [12]:
# Create datasets and loaders
batch_size = 64
train_dataset = CervicalCancerDataset(train_df, transform)
valid_dataset = CervicalCancerDataset(valid_df, transform)
test_dataset = CervicalCancerDataset(test_df, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.resnet101(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 5)
        )
    
    def forward(self, x):
        return self.base_model(x)

In [13]:
# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)


c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
# Training function
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=10):
    best_val_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as tepoch:
            for images, labels in tepoch:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                tepoch.set_postfix(loss=running_loss/total, accuracy=correct/total)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)
        
        model.eval()
        valid_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        valid_loss /= len(valid_loader.dataset)
        valid_acc = correct / len(valid_loader.dataset)
        print(f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_acc:.4f}')
        
        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            torch.save(model.state_dict(), 'cervical_cancer_model.pth')
            print("Model saved!")


In [15]:
# Train the model
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=10)

Epoch 1/10: 100%|██████████| 274/274 [02:29<00:00,  1.83batch/s, accuracy=0.864, loss=0.421]


Validation Loss: 0.2019, Validation Accuracy: 0.9349
Model saved!


Epoch 2/10: 100%|██████████| 274/274 [02:04<00:00,  2.20batch/s, accuracy=0.931, loss=0.202]


Validation Loss: 0.1407, Validation Accuracy: 0.9563
Model saved!


Epoch 3/10: 100%|██████████| 274/274 [02:20<00:00,  1.95batch/s, accuracy=0.946, loss=0.154]


...

Validation Loss: 0.0390, Validation Accuracy: 0.9877
Model saved!


Epoch 9/10: 100%|██████████| 274/274 [02:07<00:00,  2.15batch/s, accuracy=0.983, loss=0.0556]


Validation Loss: 0.0380, Validation Accuracy: 0.9885
Model saved!


Epoch 10/10: 100%|██████████| 274/274 [02:41<00:00,  1.70batch/s, accuracy=0.983, loss=0.0526]


Validation Loss: 0.0293, Validation Accuracy: 0.9928
Model saved!


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from torchvision import models
import torch.nn.functional as F
from torch.autograd import Variable
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# Apply Grad-CAM
def apply_gradcam(model, dataloader):
    target_layer = model.base_model.layer4[-1]
    cam = GradCAM(model=model, target_layers=[target_layer], use_cuda=torch.cuda.is_available())
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        targets = [ClassifierOutputTarget(label) for label in labels]
        grayscale_cam = cam(input_tensor=inputs, targets=targets)
        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(inputs[0].permute(1, 2, 0).cpu().numpy(), grayscale_cam, use_rgb=True)
        plt.imshow(visualization)
        plt.show()
        break

apply_gradcam(model, test_loader)

Exception ignored in: <function BaseCAM.__del__ at 0x000002375984F130>
Traceback (most recent call last):
  File "c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\pytorch_grad_cam\base_cam.py", line 189, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


TypeError: GradCAM.__init__() got an unexpected keyword argument 'use_cuda'